In [1]:
# default_exp protein_intensity_estimation

In [2]:
%reload_ext autoreload

%autoreload 2

In [3]:
#export
import pandas as pd
import numpy as np
import directlfq.normalization as lfqnorm
import multiprocessing
import functools

def estimate_protein_intensities(normed_df, min_nonan, num_samples_quadratic):
    "derives protein pseudointensities from between-sample normalized data"
    
    allprots = normed_df.index.get_level_values(0).unique()
    
    list_of_tuple_w_protein_profiles_and_shifted_peptides = get_list_of_tuple_w_protein_profiles_and_shifted_peptides(allprots, normed_df, num_samples_quadratic, min_nonan)
    protein_df2 = get_protein_dataframe_from_list_of_protein_profiles(allprots=allprots, list_of_tuple_w_protein_profiles_and_shifted_peptides=list_of_tuple_w_protein_profiles_and_shifted_peptides, normed_df= normed_df)
    ion_df2 = get_ion_intensity_dataframe_from_list_of_shifted_peptides(list_of_tuple_w_protein_profiles_and_shifted_peptides)

    return protein_df2, ion_df2



def get_list_of_tuple_w_protein_profiles_and_shifted_peptides(allprots, normed_df, num_samples_quadratic, min_nonan):
    multiprocessing.freeze_support()
    pool = multiprocessing.Pool()
    print(f"using {pool._processes} processes")
    list_of_tuple_w_protein_profiles_and_shifted_peptides = pool.map(functools.partial(calculate_peptide_and_protein_intensities,  allprots = allprots,normed_df = normed_df,
    num_samples_quadratic = num_samples_quadratic, min_nonan = min_nonan),range(len(allprots)))
    pool.close()

    return list_of_tuple_w_protein_profiles_and_shifted_peptides

def get_ion_intensity_dataframe_from_list_of_shifted_peptides(list_of_tuple_w_protein_profiles_and_shifted_peptides):
    ion_ints = [x[1] for x in list_of_tuple_w_protein_profiles_and_shifted_peptides]
    ion_df = 2**pd.concat(ion_ints)
    ion_df = ion_df.replace(np.nan, 0)
    return ion_df
    

def get_protein_dataframe_from_list_of_protein_profiles(allprots, list_of_tuple_w_protein_profiles_and_shifted_peptides, normed_df):
    index_list = []
    profile_list = []

    list_of_protein_profiles = [x[0] for x in list_of_tuple_w_protein_profiles_and_shifted_peptides]
    
    for idx in range(len(allprots)):
        if list_of_protein_profiles[idx] is None:
            continue
        index_list.append(allprots[idx])
        profile_list.append(list_of_protein_profiles[idx])
    
    index_for_protein_df = pd.Index(data=index_list, name="protein")
    protein_df = 2**pd.DataFrame(profile_list, index = index_for_protein_df, columns = normed_df.columns)
    protein_df = protein_df.replace(np.nan, 0)
    return protein_df


def calculate_peptide_and_protein_intensities(idx, allprots,normed_df , num_samples_quadratic, min_nonan):
    if(idx%100 ==0):
        print(f"prot {idx} of {len(allprots)}")
    protein = allprots[idx]
    peptide_intensity_df = pd.DataFrame(normed_df.loc[protein]).copy()#DataFrame definition to avoid pandas Series objects
    peptide_intensity_df = ProtvalCutter(peptide_intensity_df, maximum_df_length=100).get_dataframe()
    summed_pepint = np.nansum(2**peptide_intensity_df)

    if(peptide_intensity_df.shape[1]<2):
        shifted_peptides = peptide_intensity_df
    else:
        shifted_peptides = lfqnorm.NormalizationManagerProtein(peptide_intensity_df, num_samples_quadratic = num_samples_quadratic).complete_dataframe
    
    protein_profile = get_protein_profile_from_shifted_peptides(shifted_peptides, summed_pepint, min_nonan)
    
    return protein_profile, shifted_peptides
    

In [4]:
#export
def get_protein_profile_from_shifted_peptides(normalized_peptide_profile_df, summed_pepints, min_nonan):
    intens_vec = get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan)
    intens_vec = np.array(intens_vec)
    summed_intensity = np.nansum(2**intens_vec)
    if summed_intensity == 0: #this means all elements in intens vec are nans
        return None
    intens_conversion_factor = summed_pepints/summed_intensity
    scaled_vec = intens_vec+np.log2(intens_conversion_factor)
    return scaled_vec

def get_list_with_protein_value_for_each_sample(normalized_peptide_profile_df, min_nonan):
    intens_vec = []
    for sample in normalized_peptide_profile_df.columns:
        reps = normalized_peptide_profile_df.loc[:,sample].to_numpy()
        nonan_elems = sum(~np.isnan(reps))
        if(nonan_elems>=min_nonan):
            intens_vec.append(np.nanmedian(reps))
        else:
            intens_vec.append(np.nan)
    return intens_vec


In [5]:
#export
import pandas as pd
from numba import njit

class ProtvalCutter():
    def __init__(self, protvals_df, maximum_df_length = 100):
        self._protvals_df = protvals_df
        self._maximum_df_length = maximum_df_length
        self._dataframe_too_long = None
        self._sorted_idx = None
        self._check_if_df_too_long_and_sort_index_if_so()


    def _check_if_df_too_long_and_sort_index_if_so(self):
        self._dataframe_too_long =len(self._protvals_df.index)>self._maximum_df_length
        if self._dataframe_too_long:
            self._determine_nansorted_df_index()

    def _determine_nansorted_df_index(self):
        idxs = self._protvals_df.index
        self._sorted_idx =  sorted(idxs, key= lambda idx : self._get_num_nas_in_row(self._protvals_df.loc[idx].to_numpy()))
        
    @staticmethod
    @njit
    def _get_num_nas_in_row(row):
        return sum(np.isnan(row))


    def get_dataframe(self):
        if self._dataframe_too_long:
            return self._get_shortened_dataframe()
        else:
            return self._protvals_df

    def _get_shortened_dataframe(self):
        shortened_index = self._sorted_idx[:self._maximum_df_length]
        return self._protvals_df.loc[shortened_index]



### Unit Tests

#### Classes for testcase generation

In [6]:
import numpy as np
import pandas as pd

from  numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

class ProteinProfileGenerator():
    def __init__(self, peptide_profiles):
        self._peptide_profiles = peptide_profiles
        
        self.protein_profile_dataframe = None
        self._generate_protein_profile_dataframe()

    def _generate_protein_profile_dataframe(self):
        collected_profiles = [x.peptide_profile_vector for x in self._peptide_profiles]
        protnames_for_index = [x.protein_name for x in self._peptide_profiles]
        pepnames_for_index = [f'{idx}' for idx in range(len(self._peptide_profiles))]
        self.protein_profile_dataframe = pd.DataFrame(collected_profiles,index=[protnames_for_index, pepnames_for_index])
        self.protein_profile_dataframe = np.log2(self.protein_profile_dataframe.replace(0, np.nan))



class PeptideProfile():
    def __init__(self, protein_name, fraction_zeros_in_profile, systematic_peptide_shift, add_noise, num_samples = 20, min_intensity = 1e6, max_intensity = 1e10):


        self._fraction_zeros_in_profile = fraction_zeros_in_profile
        self._systematic_peptide_shift = systematic_peptide_shift
        self._add_noise = add_noise
        self._min_intensity = min_intensity
        self._max_intensity = max_intensity
        self._num_samples = num_samples

        self.protein_name = protein_name
        self.peptide_profile_vector = []
        self._define_peptide_profile_vector()

    def _define_peptide_profile_vector(self):
        self.peptide_profile_vector = self._get_single_peptide_profile_template()
        self._scale_profile_vector()
        if self._add_noise:
            self._apply_poisson_noise_to_profilevector()
        self._add_zeros_to_profilevector()

    def _get_single_peptide_profile_template(self):
        rs = RandomState(MT19937(SeedSequence(42312)))
        return rs.randint(low=self._min_intensity, high=self._max_intensity,size=self._num_samples)

    def _scale_profile_vector(self):
        self.peptide_profile_vector = self.peptide_profile_vector*self._systematic_peptide_shift

    def _apply_poisson_noise_to_profilevector(self):
        self.peptide_profile_vector = np.random.poisson(lam=self.peptide_profile_vector, size=len(self.peptide_profile_vector))

    def _add_zeros_to_profilevector(self):
        num_elements_to_set_zero = int(self._num_samples*self._fraction_zeros_in_profile)
        idxs_to_set_zero = np.random.choice(self._num_samples,size=num_elements_to_set_zero, replace=False)
        self.peptide_profile_vector[idxs_to_set_zero] = 0
        


#### Tests

In [7]:
import pandas as pd
import numpy as np
#test df cutting

def test_sorting_by_num_nans():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['P', 'P', 'P'],['A', 'B', 'C']])
    pcutter = ProtvalCutter(df,maximum_df_length=2)
    sorted_idx = pcutter._sorted_idx
    df_sorted = df.loc[sorted_idx]
    display(df)
    display(df_sorted)
    assert np.allclose(df_sorted.iloc[2].to_numpy(), vals1,equal_nan=True)
    assert np.allclose(df_sorted.iloc[0].to_numpy(), vals3,equal_nan=True)
    

def test_cutting_of_df():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C']])
    pcutter = ProtvalCutter(df, maximum_df_length=2)
    cut_df = pcutter.get_dataframe()
    ion_idx = [x[0] for x in cut_df.index]
    print(ion_idx)
    assert ion_idx == ['C', 'B']





test_sorting_by_num_nans()
test_cutting_of_df()

0    1    2   3
P A  9.0  NaN  NaN NaN
  B  5.0  6.0  NaN NaN
  C  1.0  2.0  3.0 NaN

0    1    2   3
P C  1.0  2.0  3.0 NaN
  B  5.0  6.0  NaN NaN
  A  9.0  NaN  NaN NaN

['C', 'B']


In [8]:
def test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=False)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=3, add_noise=False)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=0.1, add_noise=False)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=100, add_noise=False)
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,11].dropna().to_numpy()
    display(column_from_shifted)
    assert np.allclose(column_from_shifted, column_from_shifted[0])
    
test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other()



0          1          2          3          4          5   \
protA 0       NaN  44.474241  41.489085  43.216307  43.505708  42.338404   
      1       NaN        NaN        NaN        NaN        NaN  32.372620   
      2  29.73029  29.601566  26.616410  28.343632        NaN  27.465729   
      3       NaN        NaN        NaN        NaN        NaN        NaN   

                6          7          8          9          10         11  \
protA 0  43.454211  43.102621  41.897534  43.625508  43.261340  40.203349   
      1        NaN        NaN        NaN        NaN        NaN        NaN   
      2  28.581537  28.229946  27.024859  28.752833  28.388665  25.330674   
      3        NaN  38.195731        NaN  38.718617        NaN        NaN   

                12         13         14         15         16         17  \
protA 0  44.447364  44.554027  44.692343  44.701690  43.106703  44.630975   
      1        NaN        NaN        NaN  34.735906        NaN        NaN   
      2  29.574689  29.681352  29.819668  29.829015        NaN  29.758301   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                18         19  
protA 0  43.380650        NaN  
      1        NaN        NaN  
      2  28.507975  28.534739  
      3        NaN        NaN

0          1          2          3          4         5   \
protA 0        NaN  34.508456  31.523301  33.250522  33.539923  32.37262   
      1        NaN        NaN        NaN        NaN        NaN  32.37262   
      2  34.637181  34.508456  31.523301  33.250522        NaN  32.37262   
      3        NaN        NaN        NaN        NaN        NaN       NaN   

                6          7         8          9          10         11  \
protA 0  33.488427  33.136837  31.93175  33.659723  33.295555  30.237564   
      1        NaN        NaN       NaN        NaN        NaN        NaN   
      2  33.488427  33.136837  31.93175  33.659723  33.295555  30.237564   
      3        NaN  38.195731       NaN  38.718617        NaN        NaN   

                12         13         14         15         16         17  \
protA 0  34.481579  34.588243  34.726559  34.735906  33.140919  34.665191   
      1        NaN        NaN        NaN  34.735906        NaN        NaN   
      2  34.481579  34.588243  34.726559  34.735906        NaN  34.665191   
      3        NaN        NaN        NaN        NaN        NaN        NaN   

                18         19  
protA 0  33.414865        NaN  
      1        NaN        NaN  
      2  33.414865  33.441629  
      3        NaN        NaN

array([30.23756424, 30.23756424])

In [9]:
def test_that_profiles_with_noise_are_close():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    normed_ion_profile = lfqnorm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,9].dropna().to_numpy()

    assert np.allclose(column_from_shifted, column_from_shifted[0],rtol=0.01, atol=0.01)


test_that_profiles_with_noise_are_close()

0          1          2          3          4          5   \
protA 0  44.602965  44.474241  41.489086  43.216306  43.505708  42.338405   
      1  44.602973  44.474245  41.489093  43.216303  43.505712  42.338374   
      2  44.602927  44.474167  41.489203  43.216121  43.505719  42.338147   
      3  44.602966  44.474242  41.489086  43.216306  43.505708  42.338405   

                6          7          8          9          10         11  \
protA 0  43.454211  43.102621  41.897534  43.625507  43.261340  40.203348   
      1  43.454193  43.102599  41.897565  43.625508  43.261340  40.203328   
      2  43.453983  43.102657  41.897513  43.625575  43.261387  40.203133   
      3  43.454210  43.102617  41.897541  43.625506  43.261340  40.203350   

                12         13         14         15         16         17  \
protA 0  44.447364  44.554027  44.692343  44.701689  43.106703  44.630975   
      1  44.447366  44.554016  44.692351  44.701687  43.106688  44.630973   
      2  44.447396  44.554021  44.692401  44.701686  43.106710  44.630943   
      3  44.447362  44.554027  44.692343  44.701691  43.106709  44.630974   

                18         19  
protA 0  43.380650  43.407413  
      1  43.380648  43.407424  
      2  43.380651  43.407527  
      3  43.380649  43.407413

In [10]:
import directlfq.protein_intensity_estimation as intensity_estimation

def test_that_protein_intensities_are_retained():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    
    peptide_profiles = [peptide1, peptide2, peptide3, peptide4]
    summed_intensity_protein = sum([np.nansum(x.peptide_profile_vector) for x in peptide_profiles])
    
    protein_df = ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100)
    display(protein_df_normed)
    summed_lfq_intensities = np.sum(protein_df_normed.iloc[0].to_numpy())
    assert np.allclose(summed_lfq_intensities, summed_intensity_protein)


test_that_protein_intensities_are_retained()

using 10 processes
prot 0 of 1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.459300e+13,2.249371e+13,2.840800e+12,9.405541e+12,1.149496e+13,5.118116e+12,1.109181e+13,8.692853e+12,3.770471e+12,1.249023e+13,9.703790e+12,1.165194e+12,2.207859e+13,2.377277e+13,2.616480e+13,2.633489e+13,8.717469e+12,2.507516e+13,1.054042e+13,1.073779e+13


In [11]:
import directlfq.protein_intensity_estimation as intensity_estimation

def run_with_multiple_proteins():
    peptide1= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= PeptideProfile(protein_name="protB",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide5= PeptideProfile(protein_name="protC",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide6= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide7= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide8= PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    peptide_profiles = [peptide1, peptide2, peptide3, peptide4, peptide5, peptide6, peptide7, peptide8]
    protein_df = ProteinProfileGenerator(peptide_profiles).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100)
    display(protein_df_normed)
    
run_with_multiple_proteins()

using 10 processes
prot 0 of 4


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
protein,,,,,,,,,,,,,,,,,,,,
protA,2.366230e+13,2.164219e+13,2.733293e+12,9.049255e+12,1.105986e+13,4.924507e+12,1.067212e+13,8.363891e+12,3.627820e+12,1.201748e+13,9.336531e+12,1.121090e+12,2.124302e+13,2.287311e+13,2.517442e+13,2.533815e+13,8.387594e+12,2.412596e+13,1.014153e+13,1.033142e+13
protB,8.906529e+11,8.146283e+11,1.028819e+11,3.406311e+11,4.162947e+11,1.853564e+11,4.016970e+11,3.148184e+11,1.365503e+11,4.523398e+11,3.514303e+11,4.219801e+10,7.995923e+11,8.609485e+11,9.475767e+11,9.537363e+11,3.157099e+11,9.081145e+11,3.817290e+11,3.888775e+11
protC,8.906529e+11,8.146278e+11,1.028816e+11,3.406298e+11,4.162955e+11,1.853567e+11,4.016983e+11,3.148188e+11,1.365501e+11,4.523403e+11,3.514312e+11,4.219800e+10,7.995907e+11,8.609473e+11,9.475769e+11,9.537378e+11,3.157110e+11,9.081163e+11,3.817291e+11,3.888764e+11
protD,2.671959e+12,2.443881e+12,3.086458e+11,1.021892e+12,1.248886e+12,5.560684e+11,1.205094e+12,9.444566e+11,4.096503e+11,1.357021e+12,1.054290e+12,1.265945e+11,2.398776e+12,2.582847e+12,2.842729e+12,2.861203e+12,9.471356e+11,2.724347e+12,1.145186e+12,1.166629e+12


## Learning tests

In [ ]:
import pandas as pd
import numpy as np

def test_that_dataframe_is_generated_as_expected():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=['A', 'A', 'A'])
    display(df)
    assert df.iloc[2, 2] == 11
    assert df.iloc[1, 2] == 7



test_that_dataframe_is_generated_as_expected()

,0,1,2,3
A,1,2,3,4
A,5,6,7,8
A,9,10,11,12


In [11]:
def test_retrieval_of_numpy_arrays_from_dataframe():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C'], ['a', 'b', 'a']])
    display(df)
    assert np.allclose(vals2, df.loc['B'])
    assert np.allclose([2, 6, 10], df.loc[:,1])

test_retrieval_of_numpy_arrays_from_dataframe()

,,0,1,2,3
A,a,1,2,3,4
B,b,5,6,7,8
C,a,9,10,11,12


In [12]:
def test_setting_numpy_seed():
    from numpy.random import MT19937
    from numpy.random import RandomState, SeedSequence

    rs = RandomState(MT19937(SeedSequence(42)))
    res = rs.randint(10,size=20)
    display(res)

test_setting_numpy_seed()

array([2, 6, 8, 8, 3, 3, 3, 3, 4, 7, 2, 7, 5, 4, 0, 8, 1, 3, 7, 1])